In [13]:
import pandas as pd
from pathlib import Path
import os

if not os.getcwd().endswith('ratio24-new-cam'):
    os.chdir('../')
print(os.getcwd())

DATA_PATH = Path('data')

df_test = pd.read_csv(os.path.join(DATA_PATH, 'ru_sentences_with_objs.tsv'), sep='\t', encoding='utf-8')
df_test = df_test.rename(columns={"text": "answer", "tag": "labels"})
df_test["labels"].replace({"NONE": 0, "BETTER": 2, "WORSE": 3}, inplace=True)

/home/bondarenko/laptop/Jena/research/ACQuA/ratio-24/ratio24-new-cam


In [14]:
df_test.head()

,labels,answer,Object1,Object2,CommonObj,Aspect
0,0,Смотреть фильмы онлайн удобнее всего в браузер...,"Internet, Explorer",Firefox,NaN,NaN
1,0,"Было бы лучше, если бы я мог двигать мускулами...",люд,животн,NaN,NaN
2,0,Бренды наряду с известными в европе производит...,лондон,париж,NaN,NaN
3,3,"В спонже копятся бактерии, а глубоко промыть е...",кист,спонж,NaN,NaN
4,0,При нанесении тональника лучше использовать сп...,кист,спонж,NaN,NaN


In [15]:
import re
import string


def mask_objects_ru(row):

    object_1 = row["Object1"].translate(
        str.maketrans('', '', string.punctuation))
    object_2 = row["Object2"].translate(
        str.maketrans('', '', string.punctuation))
    text = row["answer"]

    # pattern1 = fr'\b\w*{object_1}\w*\b'
    # pattern2 = fr'\b\w*{object_2}\w*\b'
    pattern1 = fr'\b{object_1}\w*\b'
    pattern2 = fr'\b{object_2}\w*\b'

    masked_text = re.sub(pattern1, '[FIRST_ENTITY]', text, flags=re.IGNORECASE)
    masked_final = re.sub(
        pattern2, '[SECOND_ENTITY]', masked_text, flags=re.IGNORECASE)
    row["text"] = masked_final

    return row


def transform_data_ru(df):
  df = df.apply(mask_objects_ru, axis=1)
  return df

In [16]:
df_test = transform_data_ru(df_test)

# df_test.to_csv(os.path.join(DATA_PATH, 'ru_sentences_with_objs_masked.tsv'), sep='\t', encoding='utf-8', index=False)